A nb for testing and viewing the effeciveness of a models predictions 

This nb contains: 
- a way to crop and save images based on the mask 

In [11]:
##adding an integration of predict on training here to bug fix it

import os 

#make directory for current prediction run
!pwd
!CURRENTDATE=`date +"%Y-%m-%d-%T"`
!mkdir /home/lmeyers/paintDetect/masks/predict_$CURRENTDATE

image_folder = "/home/lqmeyers/paintDetect/images/training/"
true_mask_folder = "/home/lqmeyers/paintDetect/masks/training/"
pred_mask_folder = '/home/lqmeyers/paintDetect/masks/predict_2023-05-18-16:09:23/'

#use os.walk to get all files for a folder
dir_list = os.walk(image_folder)

for root, dirs, files in dir_list:
    for f in files: 
        #for all files in folder do: 
        print(f)
        !echo python ./Pytorch-UNet/predict.py -i $f  -m /home/lmeyers/paintDetect/wandb/run-20230517_224404-8xbgtqw6/files/model.pth -o /home/lmeyers/paintDetect/masks/predict_$CURRENTDATE${f:41:43}"pred.jpg"


/home/lqmeyers/paintDetect
mkdir: cannot create directory ‘/home/lmeyers/paintDetect/masks/predict_’: No such file or directory
f06x2022_06_28.mp4.track000041.frame000668.jpg
python ./Pytorch-UNet/predict.py -i f -m /home/lmeyers/paintDetect/wandb/run-20230517_224404-8xbgtqw6/files/model.pth -o /home/lmeyers/paintDetect/masks/predict_pred.jpg
f09x2022_06_28.mp4.track000321.frame005961.jpg
python ./Pytorch-UNet/predict.py -i f -m /home/lmeyers/paintDetect/wandb/run-20230517_224404-8xbgtqw6/files/model.pth -o /home/lmeyers/paintDetect/masks/predict_pred.jpg
f06x2022_06_28.mp4.track000389.frame008785.jpg
python ./Pytorch-UNet/predict.py -i f -m /home/lmeyers/paintDetect/wandb/run-20230517_224404-8xbgtqw6/files/model.pth -o /home/lmeyers/paintDetect/masks/predict_pred.jpg
f08x2022_06_28.mp4.track000087.frame002072.jpg
python ./Pytorch-UNet/predict.py -i f -m /home/lmeyers/paintDetect/wandb/run-20230517_224404-8xbgtqw6/files/model.pth -o /home/lmeyers/paintDetect/masks/predict_pred.jpg
f17x

In [22]:
import cv2

##---------file handling utils---------------------
def getPath(file):
    '''uses a path string to get just the directoy of a file'''
    strOut = ''
    i = 1
    while file[-i] != '/':
        i = i + 1
        #print(file[-i])
    strOut = file[0:len(file)-(i-1)]
    return strOut

def getName(file):
    '''uses a path string to get the name of a file'''
    strOut = ''
    i = 1
    while file[-i] != '/':
        i = i + 1
        #print(file[-i])
    strOut = file[-(i-1):]
    return strOut

#---------img operations-------------------------------

def crop_by_mask(image,mask,output_path):
    """
    takes in an image, and a corresponding binary mask and saves new file of image 
    values only on pixels where mask was positive. 

    Parameters:
    image (str): a string of path to image.
    mask (str): a string og path to mask 

    Returns:
    saves file to image_path/image_name.paint_only.png
    """
    img = cv2.imread(image)
    mask = cv2.imread(mask)
    output_path = output_path
    image_name = getName(image)

    #Inspect file reads
    #cv2.imshow('crop',mask)#result)
    #cv2.waitKey(5000)
    #cv2.destroyAllWindows()

    
    # Convert the image to grayscale
    gray = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)

    # Apply a threshold to create a binary image
    ret, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)

    # Find contours in the mask
    contours, _ = cv2.findContours(binary, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if len(contours) == 0:
        print("No contours found in the mask.")
        return

    # Find the bounding box of the largest contour
    largest_contour = max(contours, key=cv2.contourArea)
    x, y, w, h = cv2.boundingRect(largest_contour)

    # Crop the image based on the bounding box
    #cropped_image = img[y:y+h, x:x+w]

    # Create a mask of the cropped region
    #cropped_mask = mask[y:y+h, x:x+w]

    # Apply the mask to the cropped image
    #result = cv2.bitwise_and(cropped_image, cropped_image, mask=cropped_mask)
    result = cv2.bitwise_and(img,img,mask=binary)

    # Save the result to the output path
    #cv2.imwrite(output_path+image_name[:-4]+".paint_masked.png", result)
    #cv2.imshow('crop',result)
    #cv2.waitKey(5000)
    #cv2.destroyAllWindows()
    print(f"Cropped image saved to: {output_path+image_name[:-4]}.paint_masked.png")



In [20]:

# Example usage
image_path = "/home/lqmeyers/paintDetect/images/testing/f17x2022_06_28.mp4.track000206.frame006589.jpg"
true_mask_path = "/home/lqmeyers/paintDetect/masks/testing/f17x2022_06_28.mp4.track000206.frame006589.Paint.png"
#crop_by_mask(image_path,mask_path,output_path=getPath(image_path))



In [23]:
import os

#Run through a folder and save all the crops as above

image_folder = "/home/lqmeyers/paintDetect/images/training/"
true_mask_folder = "/home/lqmeyers/paintDetect/masks/training/"
pred_mask_folder = '/home/lqmeyers/paintDetect/masks/predict_2023-05-18-16:09:23/'


dir_list = os.walk(image_folder)
for root, dirs, files in dir_list:
    for f in files: 
        #print(f)
        print(root[0:-16]+'masks/training/'+f[:-4]+'.Paint.png')
        print(pred_mask_folder+f[:-4]+'.pred.jpg')
        crop_by_mask(root+f,pred_mask_folder+f[:-4]+'.pred.jpg',pred_mask_folder)



/home/lqmeyers/paintDetect/masks/training/f06x2022_06_28.mp4.track000041.frame000668.Paint.png
/home/lqmeyers/paintDetect/masks/predict_2023-05-18-16:09:23/f06x2022_06_28.mp4.track000041.frame000668.pred.jpg
Cropped image saved to: /home/lqmeyers/paintDetect/masks/predict_2023-05-18-16:09:23/f06x2022_06_28.mp4.track000041.frame000668.paint_masked.png
/home/lqmeyers/paintDetect/masks/training/f09x2022_06_28.mp4.track000321.frame005961.Paint.png
/home/lqmeyers/paintDetect/masks/predict_2023-05-18-16:09:23/f09x2022_06_28.mp4.track000321.frame005961.pred.jpg
Cropped image saved to: /home/lqmeyers/paintDetect/masks/predict_2023-05-18-16:09:23/f09x2022_06_28.mp4.track000321.frame005961.paint_masked.png
/home/lqmeyers/paintDetect/masks/training/f06x2022_06_28.mp4.track000389.frame008785.Paint.png
/home/lqmeyers/paintDetect/masks/predict_2023-05-18-16:09:23/f06x2022_06_28.mp4.track000389.frame008785.pred.jpg
Cropped image saved to: /home/lqmeyers/paintDetect/masks/predict_2023-05-18-16:09:23/f

KeyboardInterrupt: 

Below are some tools to open and walk through the files using cv2

In [ ]:
#some tools to look at the predicted masks

import cv2 
import os

#Run through a folder 

image_folder = "/home/lqmeyers/paintDetect/images/training/"
true_mask_folder = "/home/lqmeyers/paintDetect/masks/training/"
pred_mask_folder = '/home/lqmeyers/paintDetect/masks/predict_2023-05-18-16:09:23/'


#a way to get all the files in a certan folder
dir_list = os.walk(pred_mask_folder)
for root, dirs, files in dir_list:
    for f in files: 
        print(f)
        if "paint_masked" not in f:
            image = cv2.imread(root+f)
            cv2.imshow('crop',image)
            cv2.waitKey(5000)
            cv2.destroyAllWindows()
        

Comparing predicted mask and true mask

In [36]:
#--- test image paths
image = "/home/lqmeyers/paintDetect/images/training/f6.3x2022_06_20.mp4.track000353.frame012451.jpg"
mask_true = "/home/lqmeyers/paintDetect/masks/training/f6.3x2022_06_20.mp4.track000353.frame012451.Paint.png"
mask_pred = "/home/lqmeyers/paintDetect/masks/predict_2023-05-18-16:09:23/f6.3x2022_06_20.mp4.track000353.frame012451.pred.jpg"

#---- relevant imports
import cv2
import numpy as np

#--- actual function 
def compare_masks(path1,path2):
    """
    takes in the paths to two masks and overlays them to produde a 
    new image that shows based on 4 colors where the images overlap.

    Parameters:
    path1 (str): a string of path to a binary mask.
    path2 (str): a string og path to a binary mask.  

    Returns:
    saves file to output_path/image_name.compare.png
    """

    # Load the binary masks
    #mask1 = cv2.imread(path1, cv2.IMREAD_GRAYSCALE) #this causes pixelation issues
    mask1 = cv2.imread(path1)
    mask2 = cv2.imread(path2)

    # Convert the image to grayscale
    mask1 = cv2.cvtColor(mask1, cv2.COLOR_BGR2GRAY)
    mask2 = cv2.cvtColor(mask2, cv2.COLOR_BGR2GRAY)

    # Apply a threshold to create a binary image
    ret, mask1 = cv2.threshold(mask1, 127, 255, cv2.THRESH_BINARY) 
    ret, mask2 = cv2.threshold(mask2, 127, 255, cv2.THRESH_BINARY) 
  
    # Ensure that both masks have the same shape
    mask1 = cv2.resize(mask1, (mask2.shape[1], mask2.shape[0]))

    # Define colors for each overlap condition
    colors = [(0, 0, 0), (255, 0, 0), (0, 255, 0), (0, 0, 255)]  # Black, Blue, Green, Red

    # Create an empty image with 3 channels
    result = np.zeros((mask1.shape[0], mask1.shape[1], 3), dtype=np.uint8)

    # Compare the masks and assign colors based on the overlap conditions
    for i in range(mask1.shape[0]):
        for j in range(mask1.shape[1]):
            if mask1[i, j] == 0 and mask2[i, j] == 0:  # Neither mask has a pixel
                result[i, j] = colors[0]  # Black
            elif mask1[i, j] > 0 and mask2[i, j] == 0:  # Only mask1 has a pixel
                result[i, j] = colors[1]  # Blue
            elif mask1[i, j] == 0 and mask2[i, j] > 0:  # Only mask2 has a pixel
                result[i, j] = colors[2]  # Green
            else:  # Both masks have a pixel
                result[i, j] = colors[3]  # Red

    # Display the resulting image
    #cv2.imshow('Mask Comparison', result)
    #cv2.waitKey(1000)
    #cv2.destroyAllWindows()
    cv2.imwrite(path1[:-4]+".comparison.png",result)

#-------- test calling 

#compare_masks(mask_pred,mask_true)

In [37]:
import cv2 
import os

#Run through a folder 

image_folder = "/home/lqmeyers/paintDetect/images/training/"
true_mask_folder = "/home/lqmeyers/paintDetect/masks/training/"
pred_mask_folder = '/home/lqmeyers/paintDetect/masks/predict_2023-05-18-16:09:23/'


#for all files in predicted masks, compare them to their counterparts in true masks
dir_list = os.walk(pred_mask_folder)
for root, dirs, files in dir_list:
    for f in files: 
        #print(true_mask_folder+f[:-9]+".Paint.png")
        compare_masks(root+f,true_mask_folder+f[:-9]+".Paint.png")
        